# Mount GDriver
Perpare Colab environment.

In [4]:
locale = False
try:
    import google.colab
    google.colab.drive.mount('/gdrive')
except ModuleNotFoundError:
    locale = True

In [ ]:
addh = "/gdrive/My Drive"
if locale:
    addh = "/mnt/d/My Drive"
    print("[Locale] Using address head: " + addh)
else:
    print("[Colab] Using address head: " + addh)

# Prepare Input

In [ ]:
import preprocess

In [ ]:
train_char_seqs, train_tag_seqs = preprocess.load_file(addh + "/Graduation/Data/train_dict.txt")
test_char_seqs, test_tag_seqs = preprocess.load_file(addh + "/Graduation/Data/test_dict.txt")
train_token_id_seqs, train_tag_id_seqs, tag_vocab = preprocess.preprocess(train_char_seqs, train_tag_seqs)
test_token_id_seqs, test_tag_id_seqs, tag_vocab = preprocess.preprocess(test_char_seqs, test_tag_seqs)

In [ ]:
train_token_id_seqs[0]

In [ ]:
train_tag_id_seqs[0]

In [ ]:
tag_vocab

# Build Model
using BERT

In [ ]:
import keras
import keras_bert 

In [ ]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + "/Graduation/BERT/multi_cased_L-12_H-768_A-12/bert_config.json", 
    # checkpoint file
    addh + "/Graduation/BERT/multi_cased_L-12_H-768_A-12/bert_model.ckpt", 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)

In [ ]:
MAX_LEN = 356

input_token = keras.layers.Input(shape=(MAX_LEN,))
input_segment = keras.layers.Input(shape=(MAX_LEN,))

bert_output = bert_model([input_token, input_segment])

output = keras.layers.Dense(16, activation=keras.activations.softmax)(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [ ]:
model.compute_output_shape([(1, 5), (1, 5)])

In [ ]:
bert_model.summary()

In [ ]:
model.summary()

# Fine-tuning

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"]
)

In [ ]:
model.fit(
    x=train_token_id_seqs, 
    y=train_tag_id_seqs, 
    batch_size=16, 
    epochs=10,
    verbose=2,
    validation_data=(test_token_id_seqs, test_tag_id_seqs),
    shuffle=True
)